In [1]:
import numpy as np
import matplotlib.pyplot as plt
from openfermion.chem import MolecularData
from openfermion.transforms import get_fermion_operator, jordan_wigner, bravyi_kitaev
from openfermion.linalg import get_sparse_operator
from openfermion.ops import FermionOperator
from openfermionpyscf import run_pyscf
from pyscf import fci

In [2]:
#define constants
basis = "sto-3g"  #basis set
multiplicity = 1  #spin multiplicity
charge = 0        #total charge for the molecule
distance = 0.65
geometry = [("H",(0,0,0)),("H", (0,0,distance))]  #xyz coordinates for atoms
description = str(distance)  #description for the psi4 output file

molecule = MolecularData(geometry, basis, multiplicity, charge, description)

In [3]:
molecule = run_pyscf(molecule,run_scf=1,run_fci=1)

In [4]:
print("HF energy: {} (Hartree)".format(molecule.hf_energy))
print("FCI energy: {} (Hartree)".format(molecule.fci_energy))

HF energy: -1.1129965456691684 (Hartree)
FCI energy: -1.129904784322914 (Hartree)


In [5]:
print(molecule.one_body_integrals)

[[-1.30950987e+00 -6.66133815e-16]
 [-1.11022302e-16 -4.10026381e-01]]


In [6]:
print(molecule.two_body_integrals)

[[[[6.91904405e-01 3.19189120e-16]
   [2.77555756e-16 1.76318452e-01]]

  [[2.77555756e-16 1.76318452e-01]
   [6.79683914e-01 2.22044605e-16]]]


 [[[3.19189120e-16 6.79683914e-01]
   [1.76318452e-01 4.44089210e-16]]

  [[1.76318452e-01 4.44089210e-16]
   [2.22044605e-16 7.14671111e-01]]]]


In [7]:
print(molecule.get_molecular_hamiltonian())

() 0.8141187860307693
((0, 1), (0, 0)) -1.309509868464871
((1, 1), (1, 0)) -1.309509868464871
((2, 1), (2, 0)) -0.4100263808117851
((3, 1), (3, 0)) -0.4100263808117851
((0, 1), (0, 1), (0, 0), (0, 0)) 0.3459522026149021
((0, 1), (0, 1), (2, 0), (2, 0)) 0.08815922580510357
((0, 1), (1, 1), (1, 0), (0, 0)) 0.3459522026149021
((0, 1), (1, 1), (3, 0), (2, 0)) 0.08815922580510357
((0, 1), (2, 1), (0, 0), (2, 0)) 0.08815922580510357
((0, 1), (2, 1), (2, 0), (0, 0)) 0.3398419569652301
((0, 1), (3, 1), (1, 0), (2, 0)) 0.08815922580510357
((0, 1), (3, 1), (3, 0), (0, 0)) 0.3398419569652301
((1, 1), (0, 1), (0, 0), (1, 0)) 0.3459522026149021
((1, 1), (0, 1), (2, 0), (3, 0)) 0.08815922580510357
((1, 1), (1, 1), (1, 0), (1, 0)) 0.3459522026149021
((1, 1), (1, 1), (3, 0), (3, 0)) 0.08815922580510357
((1, 1), (2, 1), (0, 0), (3, 0)) 0.08815922580510357
((1, 1), (2, 1), (2, 0), (1, 0)) 0.3398419569652301
((1, 1), (3, 1), (1, 0), (3, 0)) 0.08815922580510357
((1, 1), (3, 1), (3, 0), (1, 0)) 0.339841956

In [8]:
jw_hamiltonian = jordan_wigner(get_fermion_operator(molecule.get_molecular_hamiltonian()))
print(jw_hamiltonian)

(0.037751103946454874+0j) [] +
(-0.044079612902551794+0j) [X0 X1 Y2 Y3] +
(0.044079612902551794+0j) [X0 Y1 Y2 X3] +
(0.044079612902551794+0j) [Y0 X1 X2 Y3] +
(-0.044079612902551794+0j) [Y0 Y1 X2 X3] +
(0.18601648886230607+0j) [Z0] +
(0.17297610130745106+0j) [Z0 Z1] +
(0.12584136558006326+0j) [Z0 Z2] +
(0.16992097848261506+0j) [Z0 Z3] +
(0.1860164888623061+0j) [Z1] +
(0.16992097848261506+0j) [Z1 Z2] +
(0.12584136558006326+0j) [Z1 Z3] +
(-0.2694169314163197+0j) [Z2] +
(0.17866777775953396+0j) [Z2 Z3] +
(-0.2694169314163197+0j) [Z3]


In [9]:
#テンソル積を計算するための関数
def kron_N(*ops):
    tmp = ops[0]
    for op in ops[1:]:
        tmp = np.kron(tmp,op)
    return tmp

bra0 = np.array([[1,0]])
bra1 = np.array([[0,1]])
HFbra = kron_N(bra1, bra1, bra0, bra0)
HFket = HFbra.T
print(HFbra)
jw_matrix = get_sparse_operator(jw_hamiltonian)
print(np.real(HFbra.dot(jw_matrix.dot(HFket))), molecule.hf_energy)

[[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]]
[[-1.11299655]] -1.1129965456691684


In [10]:
eigenenergies, eigenvecs = np.linalg.eigh(jw_matrix.toarray())
print(eigenenergies[0], molecule.fci_energy)

-1.129904784322913 -1.129904784322914


In [12]:
print(eigenvecs[:,0])

[ 0.        +0.j  0.        +0.j  0.        +0.j  0.09545811+0.j
  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 -0.99543345+0.j  0.        +0.j  0.        +0.j  0.        +0.j]
